# Model Consumer

In [9]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://80d04dce9402:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [11]:
from pyspark.sql.functions import from_json, col, current_timestamp
from pyspark.sql.types import StructType, FloatType, StringType
from JJAE.spark_utils import SparkUtils

# Define schema matching the producer's data
schema_base = [
    ("Close", "FloatType"),
    ("High", "FloatType"),
    ("Low", "FloatType"),
    ("Open", "FloatType"),
    ("Volume", "FloatType"),
    ("Date", "StringType"),
    ("Symbol", "StringType"),
]

schema = SparkUtils.generate_schema(schema_base)

# Read from Kafka
kafka_lines = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "8e35f34f36db:9093") \
    .option("subscribe", "kafka-apple,kafka-amazon,kafka-google") \
    .load()



In [12]:
print(schema)

StructType([StructField('Close', FloatType(), True), StructField('High', FloatType(), True), StructField('Low', FloatType(), True), StructField('Open', FloatType(), True), StructField('Volume', FloatType(), True), StructField('Date', StringType(), True), StructField('Symbol', StringType(), True)])


### Transform binary data into string

In [13]:
parsed_df = kafka_lines.selectExpr("CAST(value AS STRING) as json_string", "topic") \
    .withColumn("data", from_json(col("json_string"), schema)) \
    .select("topic", "data.*")

### Stream Visualization

In [ ]:
query = kafka_df.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", False) \
    .start()

query.awaitTermination(20)

### Save the data to parquet files

In [14]:

# Write to Parquet with simulated timestamps
query = parsed_df.writeStream \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/data/streamdata/") \
    .option("checkpointLocation", "/home/jovyan/notebooks/data/checkpoint") \
    .outputMode("append") \
    .start()

query.awaitTermination(180)

25/05/13 05:51:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 05:51:32 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


False

In [15]:
query.stop()

In [15]:
# Path to the directory where Parquet files were written
parquet_path = "/home/jovyan/notebooks/data/streamdata/"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame
parquet_df.show()

+------------+---------+---------+---------+---------+--------+----------+------+
|       topic|    Close|     High|      Low|     Open|  Volume|      Date|Symbol|
+------------+---------+---------+---------+---------+--------+----------+------+
|kafka-google|69.557945| 69.58332|67.228584|67.259926|4.6768E7|2020-01-06| GOOGL|
|kafka-google| 69.42359|69.841095| 69.24694| 69.68982| 3.433E7|2020-01-07| GOOGL|
|kafka-google| 70.65172|  71.0941|70.174995|  70.7582|  3.32E7|2020-01-09| GOOGL|
|kafka-google|71.189156| 71.78829| 71.04882| 71.65742|2.6076E7|2020-01-14| GOOGL|
|kafka-google|   72.163| 72.18987| 71.65742| 71.92862| 2.608E7|2020-01-16| GOOGL|
|kafka-google| 73.62401| 73.67526| 72.48097|72.779045|5.2424E7|2020-01-17| GOOGL|
|kafka-google|69.917725| 70.25661| 69.30018| 69.40916|3.5314E7|2020-01-08| GOOGL|
|kafka-google| 71.10803| 71.40562|70.642265|71.133415|2.6258E7|2020-01-10| GOOGL|
|kafka-google|71.658905| 71.73106| 70.92939|71.421036| 3.073E7|2020-01-13| GOOGL|
|kafka-google|  

In [16]:
sc.stop()